In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Macros

In [2]:
Cp = 1 ## sell price of each item
Cv = 0.4 ## variable cost of each item
Cs = -0.1 ## salvage value of each item, negative implies that it is actually a cost
Ch = 0.1 ## holding cost of each item
Q = 90 ## inventory level
L = 7 ## lifetime of each item
Beta = 0.8 ## discount factor for value function
Lam = 50 ## lambda for demand's poisson distribution

In [3]:
def plus(x,y):
    if x-y >= 0:
        return x-y
    else:
        return 0

## Genereate DTMC
In this step, first we need to generate a DTMC.


At each period: $X_t=(x_{t,1},x_{t,2},...x_{t,L-1})$


Need to generate $T$ periods, where $T = 10^3K, K=20,200,2000$


In the end, a array 
$$
    \left[
	\begin{array}{}
    x_{1,1} & x_{1,2} & x_{1,3} & \cdots & x_{1,L-1}\\
    x_{2,1} & x_{2,2} & x_{2,3} & \cdots & x_{2,L-1}\\
    \vdots & \vdots & \vdots & \vdots & \vdots\\    
    x_{T,1} & x_{T,2} & x_{T,3} & \cdots & x_{T,L-1}\\
	\end{array}
	\right]
$$

should be gerenated, and $x_{i,j}$ means the number of items in time i with j life-period remaining

In [4]:
def meet_demand(x_last,demand):
    '''
    use current inventory to meet the demand
    input:
    x_last is the inventory as last period, which is a numpy array with shape (L-1)
    d is the demand of this period, which is a int
    output:
    x_new is the invertory after meet the demand, which is a numpy array with shape (L-1)
    profit is the profit when meet the demand, which is a int
    '''
    
    unmet = demand ## left demand still not met by old items
    oldest = 1 ## the remaining lifetime of items to be used to satisfy the demand, which should be the oldest
    profit = 0 ## profit made during this process
    
    while (unmet > 0):
        ## if all available items is used up
        if oldest > x_last.shape[0]:
            break
        
        oldest_items = x_last[oldest-1] ## index should be lifetime-1
        
        ## if unmet demand is satisfied by the current oldest items
        if (oldest_items > unmet):
            profit += Cp*unmet ## make profit
            oldest_items -= unmet ## oldest_items get sold
            unmet = 0 ## no demand unmet by now
            x_last[oldest-1] = oldest_items ## update the vale of oldest items
        ## if not satisfied
        else:
            profit += Cp*oldest_items ## make profit
            unmet -= oldest_items ## update the unmet demand
            oldest_items = 0 ## all oldest_items get sold
            x_last[oldest-1] = 0 ## update the vale of oldest items
            oldest += 1 ## use the next current oldest to satisfy
    
    return x_last, profit

In [5]:
def replenish(x_end):
    '''
    replenish items to reach the invertory level Q
    input: 
    x_end is the inventory at the end of the last period (after meeting the demand), which is a numpy array with shape (L-1)
    output:
    x_begin is the inventory at the beginning of next period (order from last period arrives), which is a numpy array with shape (L-1)
    profit is the profit made during this process, which is a int
    '''
    profit = 0
    
    ## order new items
    order = Q-x_end[-1]
#     order = Q-np.sum(x_end[1:])
    profit -= Cv*order
    ## discard items with 1 lifetime
    x_new = x_end[1:]
    profit += Cs*x_end[0]
    
    x_new = np.append(x_new, order)
    return x_new, profit

In [16]:
def generate_DTMC(T=20000):
    '''
    generate a DTMC of 1000*K periods
    input:
    T, how many periods in this DTMC
    output:
    DTMC, which is a numpy array with shape (1000*K, L-1)
    '''
    DTMC = np.zeros((1,L-1))
    for period in range(T):
        x_last = DTMC[period]
        demand = np.random.poisson(lam=Lam)
#         print("demand is: ",demand)
        x_new, _ = meet_demand(x_last,demand)
        x_new, _  = replenish(x_new)
        DTMC = np.concatenate([DTMC, x_new.reshape((1,L-1))], axis=0)
    
    return DTMC

In [17]:
DTMC = generate_DTMC()

In [18]:
DTMC

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 32.],
       [ 0.,  0.,  0.,  0.,  0., 34.],
       ...,
       [ 0.,  0.,  0.,  0.,  0., 45.],
       [ 0.,  0.,  0.,  0.,  0., 26.],
       [ 0.,  0.,  0.,  0., 26., 64.]])

In [35]:
DTMC.shape

(20001, 6)

## Sample DTMC
$\bar x$ is the sample from DTMC where
$$\bar x = \{\bar x_1, \bar x_2, \bar x_3, ...\bar x_k \}, \bar x_i = x_{1000*i}$$


The result is a matrix:
$$
    \left[
	\begin{array}{}
    x_{1000,1} & x_{1000,2} & x_{1000,3} & \cdots & x_{1000,L-1}\\
    x_{2000,1} & x_{2000,2} & x_{2000,3} & \cdots & x_{2000,L-1}\\
    \vdots & \vdots & \vdots & \vdots & \vdots\\    
    x_{1000K,1} & x_{1000K,2} & x_{1000K,3} & \cdots & x_{1000K,L-1}\\
	\end{array}
	\right]
$$

In [31]:
def sample_DTMC(DTMC, K):
    x_sample = DTMC[0]
    x_sample = x_sample.reshape((1,L-1))
    for i in range(1000):
        x_next = DTMC[i*K].reshape(1,L-1)
        x_sample = np.concatenate([x_sample, x_next], axis=0)
    return x_sample[1:]

In [32]:
sample = sample_DTMC(DTMC, 20)

In [34]:
sample.shape

(1000, 6)

## Monte Carlo
To implement Monte Carlo method, first need to calcute a episode:
$$episode(i)=\sum_{n=0}^L \beta^n g(X_n)$$


Then average N episodes: 
$$\hat v(x) = \frac{1}{N} \sum_{i=1}^N episode(i)$$

In [38]:
!git add .

The file will have its original line endings in your working directory
